# Experiment 067: Data-Type-Specific Features

Based on exp_066 findings:
- Fragprints help single solvent (5.7% improvement)
- Fragprints hurt mixture predictions (2.4x worse)

Approach:
- Single Solvent: Use Fragprints
- Full Data: Don't use Fragprints
- Use correct RAMP NUM-based CV for full data (87 folds)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

DATA_PATH = '/home/data'

Using device: cuda


In [2]:
# Load data
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)
FRAGPRINTS_DF = pd.read_csv(f'{DATA_PATH}/fragprints_lookup.csv', index_col=0)

# Filter to non-zero variance
drfp_variance = DRFP_DF.var()
DRFP_FILTERED = DRFP_DF[drfp_variance[drfp_variance > 0].index.tolist()]

frag_variance = FRAGPRINTS_DF.var()
FRAGPRINTS_FILTERED = FRAGPRINTS_DF[frag_variance[frag_variance > 0].index.tolist()]

print(f'Spange: {SPANGE_DF.shape}')
print(f'DRFP filtered: {DRFP_FILTERED.shape}')
print(f'ACS PCA: {ACS_PCA_DF.shape}')
print(f'Fragprints filtered: {FRAGPRINTS_FILTERED.shape}')

Spange: (26, 13)
DRFP filtered: (24, 122)
ACS PCA: (24, 5)
Fragprints filtered: (24, 144)


In [3]:
# Data loading functions
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%", "RAMP NUM"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    """Leave-One-Ramp-Out CV using RAMP NUM (87 folds)"""
    for ramp in sorted(X["RAMP NUM"].unique()):
        mask = X["RAMP NUM"] != ramp
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [4]:
# Featurizer - different for single vs full
class FullFeaturizer:
    def __init__(self, mixed=False, include_fragprints=False):
        self.mixed = mixed
        self.include_fragprints = include_fragprints
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.frag_df = FRAGPRINTS_FILTERED
        
        base_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]
        if include_fragprints:
            self.feats_dim = base_dim + self.frag_df.shape[1]
        else:
            self.feats_dim = base_dim

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            
            if flip:
                X_spange = B_spange * (1-pct) + A_spange * pct
                X_drfp = B_drfp * (1-pct) + A_drfp * pct
                X_acs = B_acs * (1-pct) + A_acs * pct
            else:
                X_spange = A_spange * (1-pct) + B_spange * pct
                X_drfp = A_drfp * (1-pct) + B_drfp * pct
                X_acs = A_acs * (1-pct) + B_acs * pct
            
            if self.include_fragprints:
                A_frag = self.frag_df.loc[X["SOLVENT A NAME"]].values
                B_frag = self.frag_df.loc[X["SOLVENT B NAME"]].values
                if flip:
                    X_frag = B_frag * (1-pct) + A_frag * pct
                else:
                    X_frag = A_frag * (1-pct) + B_frag * pct
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs, X_frag])
            else:
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
            
            if self.include_fragprints:
                X_frag = self.frag_df.loc[X["SOLVENT NAME"]].values
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs, X_frag])
            else:
                return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

print('Featurizer defined')

Featurizer defined


In [5]:
# Simple Featurizer for GP (Spange + Arrhenius only)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1) / 100.0
            if flip:
                X_spange = B_spange * (1-pct) + A_spange * pct
            else:
                X_spange = A_spange * (1-pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print('Simple featurizer defined')

Simple featurizer defined


In [6]:
# GP Wrapper
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SimpleFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_feat)
        preds = np.column_stack([m.predict(X_scaled) for m in self.models])
        return torch.tensor(preds, dtype=torch.double)

print('GP wrapper defined')

GP wrapper defined


In [7]:
# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super().__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class WeightedMLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', include_fragprints=False):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'), include_fragprints=include_fragprints)
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            model = MLPModelInternal(input_dim, self.hidden_dims).double().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=20, factor=0.5)
            criterion = nn.HuberLoss()
            
            dataset = TensorDataset(X_all.to(device), y_all.to(device))
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0
                for X_batch, y_batch in loader:
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                scheduler.step(epoch_loss)
            
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize_torch(X_test, flip=False).to(device)
        preds = []
        for model in self.models:
            model.eval()
            with torch.no_grad():
                preds.append(model(X_feat).cpu())
        return torch.mean(torch.stack(preds), dim=0)

print('MLP ensemble defined')

MLP ensemble defined


In [8]:
# LGBM Wrapper
class LGBMWrapper:
    def __init__(self, data='single', include_fragprints=False):
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'), include_fragprints=include_fragprints)
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_feat = self.featurizer.featurize(X_test, flip=False)
        preds = np.column_stack([m.predict(X_feat) for m in self.models])
        return torch.tensor(preds, dtype=torch.double)

print('LGBM wrapper defined')

LGBM wrapper defined


In [9]:
# GP + MLP + LGBM Ensemble with data-type-specific features
class GPMLPLGBMEnsemble:
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30):
        self.data_type = data
        # Use Fragprints for single solvent, not for full data
        include_fragprints = (data == 'single')
        
        self.gp = GPWrapper(data=data)
        self.mlp = WeightedMLPEnsemble(hidden_dims=[32, 16], n_models=5, data=data, include_fragprints=include_fragprints)
        self.lgbm = LGBMWrapper(data=data, include_fragprints=include_fragprints)
        self.weights = {'gp': gp_weight, 'mlp': mlp_weight, 'lgbm': lgbm_weight}

    def train_model(self, X_train, y_train):
        self.gp.train_model(X_train, y_train)
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('Ensemble defined with data-type-specific features:')
print('  - Single Solvent: WITH Fragprints')
print('  - Full Data: WITHOUT Fragprints')

Ensemble defined with data-type-specific features:
  - Single Solvent: WITH Fragprints
  - Full Data: WITHOUT Fragprints


In [10]:
# Run Single Solvent CV
print('Running Single Solvent CV (with Fragprints)...')
print('='*60)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)

all_preds_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

all_preds_single = np.vstack(all_preds_single)
all_actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((all_preds_single - all_actuals_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f} (n={len(all_preds_single)})')

Running Single Solvent CV (with Fragprints)...


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:37<14:15, 37.21s/it]

  8%|▊         | 2/24 [01:12<13:17, 36.26s/it]

 12%|█▎        | 3/24 [01:46<12:14, 34.97s/it]

...

 92%|█████████▏| 22/24 [13:17<01:13, 36.82s/it]

 96%|█████████▌| 23/24 [13:52<00:36, 36.36s/it]

100%|██████████| 24/24 [14:28<00:00, 36.23s/it]

100%|██████████| 24/24 [14:28<00:00, 36.18s/it]


Single Solvent MSE: 0.008217 (n=656)


In [11]:
# Run Full Data CV (using RAMP NUM - 87 folds)
print('\nRunning Full Data CV (without Fragprints, 87 ramps)...')
print('='*60)

X_full, Y_full = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)

all_preds_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=87):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='full')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    all_preds_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

all_preds_full = np.vstack(all_preds_full)
all_actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((all_preds_full - all_actuals_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f} (n={len(all_preds_full)})')


Running Full Data CV (without Fragprints, 87 ramps)...


  0%|          | 0/87 [00:00<?, ?it/s]

  1%|          | 1/87 [03:33<5:06:10, 213.61s/it]

  2%|▏         | 2/87 [06:50<4:48:50, 203.89s/it]

  3%|▎         | 3/87 [10:11<4:43:39, 202.62s/it]

...

 98%|█████████▊| 85/87 [4:45:17<06:40, 200.44s/it]

 99%|█████████▉| 86/87 [4:48:37<03:20, 200.25s/it]

100%|██████████| 87/87 [4:51:54<00:00, 199.40s/it]

100%|██████████| 87/87 [4:51:54<00:00, 201.32s/it]


Full Data MSE: 0.007798 (n=1227)


In [12]:
# Calculate combined CV score
n_single = len(all_preds_single)
n_full = len(all_preds_full)

weighted_cv = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n' + '='*60)
print(f'FINAL RESULTS')
print(f'='*60)
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Weighted Combined MSE: {weighted_cv:.6f}')
print(f'\nBest baseline CV: 0.008194')
print(f'Improvement: {(0.008194 - weighted_cv) / 0.008194 * 100:.2f}%')


FINAL RESULTS
Single Solvent MSE: 0.008217 (n=656)
Full Data MSE: 0.007798 (n=1227)
Weighted Combined MSE: 0.007944

Best baseline CV: 0.008194
Improvement: 3.05%
